In [1]:
import pandas as pd
import math
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

In [ ]:
# This is path to the chromedriver. You can adjust as you need
path_chromedriver = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

chrome_options = Options()
service = Service(executable_path=path_chromedriver)